In [1]:
import pandas as pd
import numpy as np
import scipy as sc
from sklearn.linear_model import LogisticRegression, LinearRegression
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from datetime import datetime
from sklearn.metrics import *
from sklearn.ensemble import *
from sklearn.model_selection import train_test_split
# from sklearn.ensemble.partial_dependence import plot_partial_dependence

In [2]:
def run_data_pipeline(df):
    """
    Data pipeline is outlined by the following process:
        1) Sort the data by last trip date in ascending order
        2) Fill in missing ratings data with the average rating of all riders and drivers respectively
        3) Drop the remaining records that contain any sort of missing values (other than ratings)
        4) Convert last trip date from pandas object to datetime format
        5) Create a "churn" column of Boolean value to determine based on our company standards of active users
        6) Create column in Boolean value if app user's device is an iPhone
        7) Convert luxury car indicator to Boolean
        8) Create dummy variables for city
    """

    df = df.sort_values(by=['last_trip_date'])
    _fill_na_mean(df, 'avg_rating_of_driver')
    _fill_na_mean(df, 'avg_rating_by_driver')
    df.dropna(axis=0, inplace=True)
    df['last_trip_date'] = pd.to_datetime(df['last_trip_date'])
    df = _create_indicator(df, 'churn',  df['last_trip_date'] < '2014-06-01')
    df = _create_indicator(df, 'phone', df['phone'] == 'iPhone')
    df['luxury_car_user'] = df['luxury_car_user'] * 1
    df = pd.get_dummies(df, columns=['city'], drop_first=False)

    return df

def create_variables(df):

    X = df.drop(columns = ['signup_date', 'last_trip_date', 'churn'])
    y = df['churn']

    return X,y


def _fill_na_mean(df, column):
    df[column].fillna(df[column].mean(), inplace=True)


def _create_indicator(df, column, condition):
    df[column] = condition
    df[column] = df[column] * 1
    return df

def predict_model(X_train, y_train, X_test, model, trees = 100):
    if model == LogisticRegression:
        model = model(max_iter=1000)
    else:
        model = model(n_estimators = trees)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    y_pred_proba = model.predict_proba(X_test)[:,1]

    return y_pred, y_pred_proba, model

def print_metrics(y_test, y_pred, model_name = 'Model'):
    print(model_name, "- Accuracy Score: ", accuracy_score(y_test, y_pred))
    print(model_name, "- Precision Score: ", precision_score(y_test, y_pred))
    print(model_name, "- Recall Score: ", recall_score(y_test, y_pred))
    print(model_name, "- F1 Score: ", f1_score(y_test, y_pred, average = 'weighted'))
    tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
    print("TP:", tp, "  ", "FN:", fn, "  ", "TN:", tn, "  ", "FP:", fp)

    return

# ROC Curve
def create_roc_curve(y_test,y_pred_probs1, y_pred_probs2, y_pred_probs3, model1, model2, model3):
    fpr1, tpr1, _ = roc_curve(y_test, y_pred_probs1)
    auc1 = roc_auc_score(y_test, y_pred_probs1)

    fpr2, tpr2, _ = roc_curve(y_test, y_pred_probs2)
    auc2 = roc_auc_score(y_test, y_pred_probs2)

    fpr3, tpr3, _ = roc_curve(y_test, y_pred_probs3)
    auc3 = roc_auc_score(y_test, y_pred_probs3)

    plt.figure(1,figsize=(12,8))
    plt.plot([0, 1], [0, 1], 'k--')
    plt.plot(fpr1, tpr1, label=f'{model1} AUC={round(auc1,3)}')
    plt.plot(fpr2, tpr2, label=f'{model2} AUC={round(auc2,3)}')
    plt.plot(fpr3, tpr3, label=f'{model3} AUC={round(auc3,3)}')
    plt.xlabel('False positive rate')
    plt.ylabel('True positive rate')
    plt.title('ROC curve')
    plt.legend(loc='best')
    plt.show()

    return

def plot_feature_importance_chart(model, X):
    feature_importance = model.feature_importances_
    sorted_idx = np.argsort(feature_importance)
    pos = np.arange(sorted_idx.shape[0]) + .5
    fig = plt.figure(figsize=(12, 6))
    plt.subplot(1, 2, 1)
    plt.barh(pos, feature_importance[sorted_idx], align='center')
    plt.yticks(pos, np.array(list(X.columns))[sorted_idx])
    plt.title('Feature Importance')
    fig.tight_layout
    plt.show()

    return


In [3]:
df0_train = pd.read_csv('../data/churn_train.csv')
df0_test = pd.read_csv('../data/churn_test.csv')

In [4]:
df_train = df0_train.copy()
df_test = df0_test.copy()

In [5]:
df_final_train = run_data_pipeline(df_train)

In [6]:
X_train, y_train = create_variables(df_final_train)

In [7]:
df_final_test = run_data_pipeline(df_test)

In [8]:
X_test, y_test = create_variables(df_final_test)

In [9]:
y_pred1, y_pred_proba1, lr_model = predict_model(X_train, y_train, X_test, LogisticRegression)
y_pred2, y_pred_proba2, rfc_model  = predict_model(X_train, y_train, X_test, RandomForestClassifier)
y_pred3, y_pred_proba3, gbc_model = predict_model(X_train, y_train, X_test, GradientBoostingClassifier)

In [ ]:
print_metrics(y_test, y_pred1)
print_metrics(y_test, y_pred2)
print_metrics(y_test,y_pred3)
create_roc_curve(y_test,y_pred_proba1, y_pred_proba2,y_pred_proba3,'LR','RF','GB')

In [ ]:
plot_feature_importance_chart(gbc_model, X_train)

In [10]:
df = X_test

In [11]:
df['churn']=y_pred3

In [12]:
df.head(10)

,avg_dist,avg_rating_by_driver,avg_rating_of_driver,avg_surge,phone,surge_pct,trips_in_first_30_days,luxury_car_user,weekday_pct,city_Astapor,city_King's Landing,city_Winterfell,churn
6209,4.14,5.0,4.601011,1.0,0,0.0,1,0,100.0,1,0,0,1
8989,5.98,5.0,5.000000,1.0,1,0.0,1,0,100.0,0,0,1,1
2680,8.05,5.0,4.601011,1.0,1,0.0,1,1,100.0,0,0,1,1
7408,9.97,5.0,5.000000,1.0,1,0.0,1,0,100.0,0,0,1,1
3859,5.15,5.0,4.601011,1.0,1,0.0,1,0,100.0,1,0,0,1
3866,7.70,5.0,4.601011,1.0,0,0.0,1,0,100.0,0,0,1,1
9206,1.10,4.0,4.000000,1.0,0,0.0,1,0,100.0,1,0,0,1
7734,0.77,5.0,3.000000,1.0,1,0.0,1,0,100.0,0,0,1,1
2225,15.14,5.0,4.601011,1.0,1,0.0,1,0,100.0,0,0,1,1
8469,5.23,5.0,5.000000,1.0,0,0.0,1,0,100.0,0,1,0,1


In [15]:
import pandas as pd
import pandasql as ps

In [14]:
pip install pandasql

     |████████████████████████████████| 1.2 MB 5.9 MB/s eta 0:00:01
  Created wheel for pandasql: filename=pandasql-0.7.3-py3-none-any.whl size=26818 sha256=6b11e9c947f47e5afe1e24723ac12478ce1364cd1043c8a6f78bfcd6a7fd852f
  Stored in directory: /Users/jl1000260404/Library/Caches/pip/wheels/63/e8/ec/75b1df467ecf57b6ececb32cb16f4e86697cbfe55cb0c51f07
Successfully built pandasql
Note: you may need to restart the kernel to use updated packages.


In [39]:
query = """

SELECT COUNT(*)
FROM df
WHERE churn = 0


"""

print(ps.sqldf(query,locals()))

   COUNT(*)
0      3292


In [40]:
query1 = """

SELECT COUNT(*)
FROM df
WHERE churn = 1


"""

print(ps.sqldf(query1,locals()))

   COUNT(*)
0      6631


In [23]:
q1 = """ 

SELECT AVG(avg_rating_by_driver)
FROM df
WHERE churn = 1

"""

print(ps.sqldf(q1,locals()))

   AVG(avg_rating_by_driver)
0                   4.792184


In [24]:
q2 = """ 

SELECT AVG(avg_rating_by_driver)
FROM df
WHERE churn = 0

"""

print(ps.sqldf(q2,locals()))

   AVG(avg_rating_by_driver)
0                    4.75711


In [42]:
q3 = """ 

SELECT AVG(surge_pct)
FROM df
WHERE churn = 1

"""

print(ps.sqldf(q3,locals()))

   AVG(surge_pct)
0        8.551742


In [26]:
q4 = """ 

SELECT AVG(surge_pct)
FROM df
WHERE churn = 0

"""

print(ps.sqldf(q4,locals()))

   AVG(surge_pct)
0        9.459143


In [33]:
q5 = """ 

SELECT AVG(weekday_pct)
FROM df
WHERE churn = 1

"""

print(ps.sqldf(q5,locals()))

   AVG(weekday_pct)
0         60.675524


In [37]:
q6 = """ 

SELECT AVG(weekday_pct)
FROM df
WHERE churn = 0

"""

print(ps.sqldf(q6,locals()))

   AVG(weekday_pct)
0         61.979253


In [47]:
q7 = """ 

SELECT Count(*)
FROM df
WHERE churn = 1 AND `city_King's Landing` = 1

"""

print(ps.sqldf(q7,locals()))

   Count(*)
0       590


In [45]:
q8 = """ 

SELECT count(*)
FROM df
WHERE churn = 0 AND `city_King's Landing` = 1

"""

print(ps.sqldf(q8,locals()))

   count(*)
0      1386


In [28]:
df.columns

Index(['avg_dist', 'avg_rating_by_driver', 'avg_rating_of_driver', 'avg_surge',
       'phone', 'surge_pct', 'trips_in_first_30_days', 'luxury_car_user',
       'weekday_pct', 'city_Astapor', 'city_King's Landing', 'city_Winterfell',
       'churn'],
      dtype='object')

In [ ]:
df.head(5).T

In [ ]:
df.head(5)

In [ ]:
#new_df = df['last_trip_date'].sort_values(ascending = False)
new = df.sort_values(by = ['last_trip_date'])

In [ ]:
#new.info()
new['avg_rating_of_driver'].fillna(new['avg_rating_of_driver'].mean(), inplace = True)
new['avg_rating_by_driver'].fillna(new['avg_rating_by_driver'].mean(), inplace = True)

In [ ]:
new.info()

In [ ]:
new.dropna(axis = 0, inplace = True)

In [ ]:
new.info()

In [ ]:
df = new.copy()

In [ ]:
df['last_trip_date'] = pd.to_datetime(df['last_trip_date'])

In [ ]:
df['churn'] = df['last_trip_date'] < '2014-06-01'

In [ ]:
df['churn'] = df['churn'] * 1
df['phone'] = df['phone'] == 'iPhone'
df['phone'] = df['phone'] * 1

In [ ]:
df = pd.get_dummies(df, columns = ['city'], drop_first = False)

In [ ]:
df['luxury_car_user'] = df['luxury_car_user'] * 1

In [ ]:
df.info()

In [ ]:
df.head(10).T

In [ ]:
df['phone'].unique()

In [ ]:
df.head(5).T

In [ ]:

X = df.drop(columns = ['signup_date','last_trip_date','churn'])


In [ ]:
y = df['churn']

In [ ]:
X_train = X

In [ ]:
y_train = y

In [ ]:
log_model = LogisticRegression(max_iter = 1000)
log_model.fit(X_train,y_train)

In [ ]:
y_pred = log_model.predict(X_test)

In [ ]:
accuracy_score(y_test, y_pred)

In [ ]:
confusion_matrix(y_test, y_pred, labels = [1,0])

In [ ]:
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()

In [ ]:
(tn, fp, fn, tp)

In [ ]:
con_matrix = np.array([[tp, fn], [fp, tn]])

In [ ]:
con_matrix

In [ ]:
log_model.coef_

In [ ]:
coeff = log_model.coef_
ind = (coeff**2).argsort()
labels = [item for item in X.columns]

In [ ]:
fig,ax = plt.subplots()
plt.axhline(y=0,color='red')
plt.plot(np.linspace(0, 11, 12), coeff.T)
ax.set_xticklabels(labels)
plt.xticks(np.linspace(0,11,12), rotation=90);

In [ ]:
fig = plt.figure(figsize=(12,8))
ax = fig.add_subplot()
x = np.linspace(0, 11, 12)
markerline, stemlines, baseline = plt.stem(x, coeff.T, '-.')
plt.setp(baseline, color='r', linewidth=2)
plt.xticks(np.linspace(0,11,12), labels,rotation=90)
plt.xlabel('Features')
plt.ylabel('Beta Coefficients')
plt.title('Feature Impact')
plt.grid(linestyle ='--',lw=0.5)
plt.show()

In [ ]:
print(classification_report(y, y_pred))

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25)

rf_model = RandomForestClassifier()
rf_model.fit(X_train, y_train)

In [ ]:
rf_model.feature_importances_

In [ ]:
y_pred = rf_model.predict(X_test)

In [ ]:
# ROC Curve
def create_ROC_curve_plot(y_test,y_pred_probs, model):
    fpr, tpr, _ = roc_curve(y_test, y_pred_probs)
    plt.figure(1)
    plt.plot([0, 1], [0, 1], 'k--')
    plt.plot(fpr, tpr, label=model)
    plt.xlabel('False positive rate')
    plt.ylabel('True positive rate')
    plt.title('ROC curve')
    plt.legend(loc='best')
    plt.show()
    return 
    

In [ ]:
y_pred, y_pred_probs = model_pred(X_train, y_train, X_test, y_test, RandomForestClassifier)

In [ ]:
y_pred_probs.shape

In [ ]:
ROC_curve_plot(y_test,y_pred_probs, 'RF')

In [ ]:
y_pred, y_pred_probs = model_pred(X_train, y_train, X_test, y_test, LogisticRegression)
ROC_curve_plot(y_test,y_pred_probs, 'LR')

In [ ]:
def ROC_curve_plots(y_test,y_pred_probs1, y_pred_probs2, y_pred_probs3, model1, model2, model3):
    fpr1, tpr1, _ = roc_curve(y_test, y_pred_probs1)
    auc1 = roc_auc_score(y_test, y_pred_probs1)
    fpr2, tpr2, _ = roc_curve(y_test, y_pred_probs2)
    auc2 = roc_auc_score(y_test, y_pred_probs2)
    fpr3, tpr3, _ = roc_curve(y_test, y_pred_probs3)
    auc3 = roc_auc_score(y_test, y_pred_probs3) 
    plt.figure(1,figsize=(12,8))
    plt.plot([0, 1], [0, 1], 'k--')
    plt.plot(fpr1, tpr1, label=f'{model1} AUC={round(auc1,3)}')
    plt.plot(fpr2, tpr2, label=f'{model2} AUC={round(auc2,3)}')
    plt.plot(fpr3, tpr3, label=f'{model3} AUC={round(auc3,3)}')
    plt.xlabel('False positive rate')
    plt.ylabel('True positive rate')
    plt.title('ROC curve')
    plt.legend(loc='best')
    plt.show()
    return 

In [ ]:
y_pred1, y_pred_probs1 = model_pred(X_train, y_train, X_test, y_test, LogisticRegression)
y_pred2, y_pred_probs2 = model_pred(X_train, y_train, X_test, y_test, RandomForestClassifier)
y_pred3, y_pred_probs3 = model_pred(X_train, y_train, X_test, y_test, GradientBoostingClassifier)

In [ ]:
ROC_curve_plots(y_test,y_pred_probs1, y_pred_probs2,y_pred_probs3, 'LR', 'RF','GB')

In [ ]:
cost_benefit = np.array([[0, 0], [-0.7, 0.3]]).T
def profit_curve(cost_benefit, predicted_probs, y_test):
    """Function to calculate list of profits based on supplied cost-benefit
    matrix and prediced probabilities of data points and thier true labels.
    Parameters
    ----------
    cost_benefit    : ndarray - 2D, with profit values corresponding to:
                                          -----------
                                          | TP | FP |
                                          -----------
                                          | FN | TN |
                                          -----------
    predicted_probs : ndarray - 1D, predicted probability for each datapoint
                                    in labels, in range [0, 1]
    labels          : ndarray - 1D, true label of datapoints, 0 or 1
    Returns
    -------
    profits    : ndarray - 1D
    thresholds : ndarray - 1D
    """
    n_obs = float(len(y_test))
    # Make sure that 1 is going to be one of our thresholds
    maybe_one = [] if 1 in predicted_probs else [1] 
    all_thresholds = maybe_one + sorted(predicted_probs, reverse=True)
    thresholds = all_thresholds[::50]
    profits = []
    for threshold in thresholds:
        y_predict = predicted_probs >= threshold
        confusion_matrix_ = confusion_matrix(y_test, y_predict, labels = [1,0])
        threshold_profit = np.sum(confusion_matrix_ * cost_benefit) / n_obs
        profits.append(threshold_profit)
    return np.array(profits), np.array(thresholds)

In [ ]:

def get_model_profits(model, cost_benefit, X_train, X_test, y_train, y_test):
    """Fits passed model on training data and calculates profit from cost-benefit
    matrix at each probability threshold.
    Parameters
    ----------
    model           : sklearn model - need to implement fit and predict
    cost_benefit    : ndarray - 2D, with profit values corresponding to:
                                          -----------
                                          | TP | FP |
                                          -----------
                                          | FN | TN |
                                          -----------
    X_train         : ndarray - 2D
    X_test          : ndarray - 2D
    y_train         : ndarray - 1D
    y_test          : ndarray - 1D
    Returns
    -------
    model_profits : model, profits, thresholds
    """
    model.fit(X_train, y_train)
    predicted_probs = model.predict_proba(X_test)[:, 1]
    profits, thresholds = profit_curve(cost_benefit, predicted_probs, y_test)

    return profits, thresholds

In [ ]:
profs, thrs = profit_curve(cost_benefit, y_pred_probs3, y_test)

In [ ]:
max_thr = thrs[np.argmax(profs)]
max_thr

In [ ]:
max_profs = np.max(profs)


In [ ]:
max_profs

In [ ]:
new_model = GradientBoostingClassifier()
new_model.fit(X_train, y_train)
#print(" done.")

#print('Convenience plot with ``partial_dependence_plots``')

features = [4,7,9,10]
fig, axs = plot_partial_dependence(new_model, X_train, features,figsize = (10, 10), 
                                   feature_names= [item for item in X.columns],
                                   n_jobs=3, grid_resolution=50)
fig.suptitle('Partial dependence plots')
plt.subplots_adjust(top=0.9, bottom = 0.2, wspace = 0.5)  # tight_layout causes overlap with suptitle

#print('Custom 3d plot via ``partial_dependence``')
#fig = plt.figure()
